# Lab 1: Connect & Explore the Knowledge Base**Estimated Time:** 4 minutes---

## Step 1: Connect to Oracle Autonomous Database

In [ ]:
import oracledb

# ============================================================
# UPDATE THESE WITH YOUR CONNECTION DETAILS
# ============================================================
DB_USER     = "citypulse"
DB_PASSWORD = "YourPasswordHere"   # <-- Change this
DB_DSN      = "your-adb-host:1522/your_service_tp.adb.oraclecloud.com"  # <-- Change this

# Connect in thin mode — no Oracle Client installation needed
connection = oracledb.connect(user=DB_USER, password=DB_PASSWORD, dsn=DB_DSN)
print(f"Connected to Oracle Database {connection.version}")
print(f"User: {DB_USER}")

In [ ]:
def run_query(sql, params=None, headers=True):
    """Run a SQL query and print results in a formatted table."""
    with connection.cursor() as cursor:
        cursor.execute(sql, params or {})
        columns = [col[0] for col in cursor.description]
        rows = cursor.fetchall()

        if headers:
            widths = [max(len(str(col)), max((len(str(row[i])) for row in rows), default=0))
                      for i, col in enumerate(columns)]
            header = " | ".join(str(col).ljust(widths[i]) for i, col in enumerate(columns))
            print(header)
            print("-" * len(header))
            for row in rows:
                print(" | ".join(str(val).ljust(widths[i]) for i, val in enumerate(row)))

        print(f"\n({len(rows)} rows)")
        return rows

print("Helper function ready.")

## Step 2: Explore the Knowledge Base

In [ ]:
print("=== KNOWLEDGE BASE OVERVIEW ===\n")
run_query("""
    SELECT category, COUNT(*) AS doc_count
    FROM city_knowledge_base
    GROUP BY category
    ORDER BY doc_count DESC
""")

In [ ]:
print("=== KNOWLEDGE BASE DOCUMENTS ===\n")
run_query("""
    SELECT doc_id,
           SUBSTR(title, 1, 55) AS title,
           category,
           LENGTH(content) AS content_length
    FROM city_knowledge_base
    ORDER BY doc_id
""")

In [ ]:
print("=== SAMPLE DOCUMENT ===\n")
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT title, category, author, content
        FROM city_knowledge_base
        WHERE title LIKE '%Vibration Anomaly%' AND category = 'Emergency Protocol'
    """)
    row = cursor.fetchone()
    print(f"Title:    {row[0]}")
    print(f"Category: {row[1]}")
    print(f"Author:   {row[2]}")
    print(f"\n{row[3][:500]}...")

## Step 3: The Keyword Search ProblemA CityPulse operator notices elevated vibration readings on a bridge sensor and searches for "bridge shaking."

In [ ]:
print("=== KEYWORD SEARCH: 'bridge shaking' ===\n")
results = run_query("""
    SELECT doc_id, title, category
    FROM city_knowledge_base
    WHERE LOWER(content) LIKE '%bridge shaking%'
""")

if len(results) == 0:
    print("\n⚠️  Zero results. The operator found nothing.")

In [ ]:
print("=== KEYWORD SEARCH: 'vibration anomaly' ===\n")
run_query("""
    SELECT doc_id, SUBSTR(title, 1, 60) AS title, category
    FROM city_knowledge_base
    WHERE LOWER(content) LIKE '%vibration anomaly%'
""")

**This is the problem vector search solves.** "Bridge shaking" and "structural vibration anomaly response protocol" mean similar things — but share no words in common. In the next lab, you'll build a search system where meaning matters more than keywords.